## Tool Calling

Before you begin, please ensure Llama Stack is installed and set up by following the [Getting Started Guide](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

In this section, we'll explore how to enhance your applications with tool calling capabilities. We'll cover:
1. Setting up and using the Brave Search API
2. Creating custom tools
3. Configuring tool prompts and safety settings

Set up your connection parameters:

In [1]:
HOST = "localhost"  # Replace with your host
PORT = 5000        # Replace with your port

In [2]:
import asyncio
import os
from typing import Dict, List, Optional
from dotenv import load_dotenv

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import (
    AgentConfig,
    AgentConfigToolSearchToolDefinition,
)

# Load environment variables
load_dotenv()

# Helper function to create an agent with tools
async def create_tool_agent(
    client: LlamaStackClient,
    tools: List[Dict],
    instructions: str = "You are a helpful assistant",
    model: str = "Llama3.2-11B-Vision-Instruct",
) -> Agent:
    """Create an agent with specified tools."""
    print("Using the following model: ", model)
    agent_config = AgentConfig(
        model=model,
        instructions=instructions,
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=tools,
        tool_choice="auto",
        tool_prompt_format="json",
        enable_session_persistence=True,
    )

    return Agent(client, agent_config)

First, create a `.env` file in your notebook directory with your Brave Search API key:

```
BRAVE_SEARCH_API_KEY=your_key_here
```


In [3]:
async def create_search_agent(client: LlamaStackClient) -> Agent:
    """Create an agent with Brave Search capability."""
    search_tool = AgentConfigToolSearchToolDefinition(
        type="brave_search",
        engine="brave",
        api_key="dummy_value"#os.getenv("BRAVE_SEARCH_API_KEY"),
    )
    
    models_response = client.models.list()
    for model in models_response:
        if model.identifier.endswith("Instruct"):
            model_name = model.llama_model
    

    return await create_tool_agent(
        client=client,
        tools=[search_tool],
        model = model_name,
        instructions="""
        You are a research assistant that can search the web.
        Always cite your sources with URLs when providing information.
        Format your responses as:

        FINDINGS:
        [Your summary here]

        SOURCES:
        - [Source title](URL)
        """
    )

# Example usage
async def search_example():
    client = LlamaStackClient(base_url=f"http://{HOST}:{PORT}")
    agent = await create_search_agent(client)

    # Create a session
    session_id = agent.create_session("search-session")

    # Example queries
    queries = [
        "What are the latest developments in quantum computing?",
        #"Who won the most recent Super Bowl?",
    ]

    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)

        response = agent.create_turn(
            messages=[{"role": "user", "content": query}],
            session_id=session_id,
        )

        async for log in EventLogger().log(response):
            log.print()

# Run the example (in Jupyter, use asyncio.run())
await search_example()

Using the following model:  Llama3.2-11B-Vision-Instruct

Query: What are the latest developments in quantum computing?
--------------------------------------------------
inference> FINDINGS:
The latest developments in quantum computing include advancements in quantum processors, quantum algorithms, and quantum error correction. Researchers have made progress in building larger and more stable quantum processors, such as Google's 53-qubit processor and IBM's 127-qubit processor (1). New quantum algorithms, like the Quantum Approximate Optimization Algorithm (QAOA), have been developed to solve complex optimization problems (2). Additionally, researchers are exploring quantum error correction techniques to mitigate errors in quantum computations (3).

SOURCES:
- "Quantum computing: a very short introduction" by Samuel J. Lomonaco (URL)
- "Quantum Approximate Optimization Algorithm" by Edward Farhi et al. (URL)
- "Quantum error correction for beginners" by Robert A. Hulet (URL)
- "Google

## 3. Custom Tool Creation

Let's create a custom weather tool:

In [4]:
from typing import TypedDict, Optional, Dict, Any
from datetime import datetime

from llama_stack_client.types.tool_param_definition_param import ToolParamDefinitionParam
from llama_stack_client.types import CompletionMessage
class WeatherTool:
    """Example custom tool for weather information."""

    def get_name(self) -> str:
        return "get_weather"

    def get_description(self) -> str:
        return "Get weather information for a location"

    def get_params_definition(self) -> Dict[str, ToolParamDefinitionParam]:
        return {
            "location": ToolParamDefinitionParam(
                param_type="str",
                description="City or location name",
                required=True
            ),
            "date": ToolParamDefinitionParam(
                param_type="str",
                description="Optional date (YYYY-MM-DD)",
                required=False
            )
        }
    async def run(self,messages):
        for message in messages:
            print("tool_calls starting: ", message.tool_calls[0])
            return_dict = message.tool_calls[0].arguments
            location = return_dict.get("location", None)
            date = return_dict.get("date", None)
            print("Using Location:",location)
            if date:
                print("Using date:",date)
            return await self.run_impl(location,date)
    
    async def run_impl(self, location: str, date: Optional[str] = None) -> Dict[str, Any]:
        """Simulate getting weather data (replace with actual API call)."""
        # Mock implementation
        return [CompletionMessage(
        content="""{
            "temperature": 72.5,
            "conditions": "partly cloudy",
            "humidity": 65.0
        }""",
        role='assistant',
        stop_reason='end_of_message',
        tool_calls=[],
    )],
        

async def create_weather_agent(client: LlamaStackClient) -> Agent:
    """Create an agent with weather tool capability."""
    models_response = client.models.list()
    for model in models_response:
        if model.identifier.endswith("Instruct"):
            model_name = model.llama_model
    agent_config = AgentConfig(
        model=model_name,
        instructions="""
        You are a weather assistant that can provide weather information.
        Always specify the location clearly in your responses.
        Include both temperature and conditions in your summaries.
        """,
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=[
            {
                "function_name": "get_weather",
                "description": "Get weather information for a location",
                "parameters": {
                    "location": {
                        "param_type": "str",
                        "description": "City or location name",
                        "required": True,
                    },
                    "date": {
                        "param_type": "str",
                        "description": "Optional date (YYYY-MM-DD)",
                        "required": False,
                    },
                },
                "type": "function_call",
            }
        ],
        tool_choice="auto",
        tool_prompt_format="json",
        input_shields=[],
        output_shields=[],
        enable_session_persistence=True
    )

    # Create the agent with the tool
    weather_tool = WeatherTool()
    agent = Agent(
        client=client,
        agent_config=agent_config,
        custom_tools=[weather_tool]
    )

    return agent

# Example usage
async def weather_example():
    client = LlamaStackClient(base_url=f"http://{HOST}:{PORT}")
    agent = await create_weather_agent(client)
    session_id = agent.create_session("weather-session")

    queries = [
        "What's the weather like in San Francisco?",
        "Tell me the weather in Tokyo tomorrow",
    ]

    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)

        response = agent.create_turn(
            messages=[{"role": "user", "content": query}],
            session_id=session_id,
        )
        
        async for log in EventLogger().log(response):
            log.print()

# For Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

# Run the example
await weather_example()


Query: What's the weather like in San Francisco?
--------------------------------------------------
inference> {
    "type": "function",
    "name": "get_weather",
    "parameters": {
        "location": "San Francisco"
    }
}
tool_calls starting:  ToolCall(arguments={'location': 'San Francisco'}, call_id='420a73c9-1f6b-4f98-8fef-a7b9d2feb753', tool_name='get_weather')
Using Location: San Francisco

Query: Tell me the weather in Tokyo tomorrow
--------------------------------------------------
inference> {"type": "function", "name": "get_weather", "parameters": {"location": "Tokyo", "date": "tomorrow"}}
tool_calls starting:  ToolCall(arguments={'location': 'Tokyo', 'date': 'tomorrow'}, call_id='9f94b5f1-fa5a-492f-a81f-dcfaafc970f3', tool_name='get_weather')
Using Location: Tokyo
Using date: tomorrow


Thanks for checking out this tutorial, hopefully you can now automate everything with Llama! :D

Next up, we learn another hot topic of LLMs: Memory and Rag. Continue learning [here](./04_Memory101.ipynb)!